In [1]:
# !pip install onnx onnxruntime
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# device = "cuda:0" 


In [2]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



In [3]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_bev')==False:
    os.mkdir('./deploy_pillar_sfaw_bev')
log_file = './deploy_pillar_sfaw_bev/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-04-11 05:05:13,600   INFO  Loading Custom dataset.
2024-04-11 05:05:13,602   INFO  Total samples for CUSTOM dataset: 0


In [4]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-04-11 05:05:15,729   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-11 05:05:15,847   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-11 05:05:15,880   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [5]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")
inputs = batch_dict
inputs

{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1}

In [6]:
model

CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [7]:
inputs_vfe = model.vfe(inputs)
inputs_3d = model.backbone_3d(inputs_vfe)
# inputs_bev = model.map_to_bev_module(inputs_3d)

# inputs_bev
inputs_3d

{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1,
 'pillar_features': tensor([[-1.0969e+00, -8.8788e-02,  1.5693e-01,  ..., -1.0477e+00,
           3.8000e-02, -1.9860e-01],
         [-1.0964e+00, -7.0831e-02,  1.6052e-01,  ..., -1.0824e+00,
           4.2364e-02, -1.9235e-01],
         [-1.0862e+00, -1.0094e-01,  1.7857e-01,  ..., -1.0550e+00,
           2.6313e-02, -1.5948e-01],
         .

In [8]:
input_names = [
    'voxel_features',
    'use_lead_xyz',
    'batch_size',
    'pillar_features',
    'voxel_coords',
]
output_names = ["output",]

In [9]:
# data_dict = {}
input_dict = {}
input_dict['batch_dict']={}
input_dict['batch_dict']['voxel_features'] = inputs_3d['voxel_features']
input_dict['batch_dict']['use_lead_xyz'] = inputs_3d['use_lead_xyz']
input_dict['batch_dict']['batch_size'] = torch.tensor(inputs_3d['batch_size'])
input_dict['batch_dict']['pillar_features'] = inputs_3d['pillar_features']
input_dict['batch_dict']['voxel_coords'] = inputs_3d['voxel_coords']
# input_dict['data_dict']['spatial_features'] = inputs_3d['spatial_features']

# data_dict.update(inputs_bev)
# data_dict
input_dict

{'batch_dict': {'voxel_features': tensor([[-1.0969e+00, -8.8788e-02,  1.5693e-01,  ..., -1.0477e+00,
            3.8000e-02, -1.9860e-01],
          [-1.0964e+00, -7.0831e-02,  1.6052e-01,  ..., -1.0824e+00,
            4.2364e-02, -1.9235e-01],
          [-1.0862e+00, -1.0094e-01,  1.7857e-01,  ..., -1.0550e+00,
            2.6313e-02, -1.5948e-01],
          ...,
          [-3.6077e-01,  2.1283e-01, -1.5834e-01,  ..., -1.2332e+00,
           -1.6347e-01,  1.6772e-01],
          [-6.1859e-01,  3.5162e-01, -1.1130e-01,  ..., -1.3723e+00,
           -1.0931e-01, -5.4504e-02],
          [-6.1340e-01,  3.2128e-01, -1.2228e-02,  ..., -1.2449e+00,
            6.9372e-04, -1.3799e-01]], device='cuda:0',
         grad_fn=<NativeLayerNormBackward0>),
  'use_lead_xyz': tensor([1.], device='cuda:0'),
  'batch_size': tensor(1),
  'pillar_features': tensor([[-1.0969e+00, -8.8788e-02,  1.5693e-01,  ..., -1.0477e+00,
            3.8000e-02, -1.9860e-01],
          [-1.0964e+00, -7.0831e-02,  1.6052e

In [10]:
with torch.no_grad():
    DSVT_2d_Backbone = model.backbone_2d
    BEV_module = model.map_to_bev_module

In [11]:
import torch
import torch.nn as nn

class ONNXModule(nn.Module):
    def __init__(self, backbone_2d, bev_module):
        super(ONNXModule, self).__init__()
        self.backbone_2d = backbone_2d
        self.bev_module = bev_module

    def forward(self, data_dict):
        data_dict = self.backbone_2d(data_dict)
        print(data_dict)
        data_dict = self.bev_module(data_dict)
        return data_dict

In [12]:
onnx_module = ONNXModule(DSVT_2d_Backbone, BEV_module)


In [13]:
base_name = "./deploy_pillar_sfaw_bev/dsvt_bev"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

# allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()
# module_o = onnx_module().eval().cuda()
torch.onnx.export(
    model.map_to_bev_module,
    input_dict,
    onnx_path, 
    input_names=input_names,
    output_names=output_names, 
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

/opt/dec/pcdet/models/backbones_2d/map_to_bev/pointpillar_scatter.py:53: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  batch_size = coords[:, 0].max().int().item() + 1
/usr/local/lib/python3.8/dist-packages/torch/onnx/utils.py:715: UserWarning: We detected that you are modifying a dictionary that is an input to your model. Note that dictionaries are allowed as inputs in ONNX but they should be handled with care. Usages of dictionaries is not recommended, and should not be used except for configuration use. Also note that the order and values of the keys must remain the same. 
  warnings.warn(warning)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
# test onnx
# ort_session = ort.InferenceSession(onnx_path)
# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
#               ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
#               ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
#               ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
#               ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
#               ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
# input_names = [
#     'voxel_features',
#     'use_lead_xyz',
#     'batch_size',
#     'pillar_features',
#     'voxel_coords',
#     'spatial_features'
# ]
# ort_inputs = {
#     'voxel_features' : to_numpy(input_dict['data_dict']['voxel_features']),
#     'use_lead_xyz' : to_numpy(input_dict['data_dict']['use_lead_xyz']),
#     'batch_size' :  np.array(1, dtype=np.int64),
#     'pillar_features' : to_numpy(input_dict['data_dict']['pillar_features']),
#     'voxel_coords' : to_numpy(input_dict['data_dict']['voxel_coords']),
#     'spatial_features' : to_numpy(input_dict['data_dict']['spatial_features'])
# }
# # = input_dict
# ort_outs = ort_session.run(None, ort_inputs)
# ort_outs[0].shape


In [16]:
input_dict

{'batch_dict': {'voxel_features': tensor([[-1.0969e+00, -8.8788e-02,  1.5693e-01,  ..., -1.0477e+00,
            3.8000e-02, -1.9860e-01],
          [-1.0964e+00, -7.0831e-02,  1.6052e-01,  ..., -1.0824e+00,
            4.2364e-02, -1.9235e-01],
          [-1.0862e+00, -1.0094e-01,  1.7857e-01,  ..., -1.0550e+00,
            2.6313e-02, -1.5948e-01],
          ...,
          [-3.6077e-01,  2.1283e-01, -1.5834e-01,  ..., -1.2332e+00,
           -1.6347e-01,  1.6772e-01],
          [-6.1859e-01,  3.5162e-01, -1.1130e-01,  ..., -1.3723e+00,
           -1.0931e-01, -5.4504e-02],
          [-6.1340e-01,  3.2128e-01, -1.2228e-02,  ..., -1.2449e+00,
            6.9372e-04, -1.3799e-01]], device='cuda:0',
         grad_fn=<NativeLayerNormBackward0>),
  'use_lead_xyz': tensor([1.], device='cuda:0'),
  'batch_size': tensor(1),
  'pillar_features': tensor([[-1.0969e+00, -8.8788e-02,  1.5693e-01,  ..., -1.0477e+00,
            3.8000e-02, -1.9860e-01],
          [-1.0964e+00, -7.0831e-02,  1.6052e

In [17]:
# test onnx
ort_session = ort.InferenceSession(onnx_path)
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
#               ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
#               ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
#               ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
#               ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
#               ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
ort_inputs = {
    'voxel_features' : to_numpy(input_dict['batch_dict']['voxel_features']),
    'use_lead_xyz' : to_numpy(input_dict['batch_dict']['use_lead_xyz']),
    'batch_size' :  np.array(1, dtype=np.int64),
    'pillar_features' : to_numpy(input_dict['batch_dict']['pillar_features']),
    'voxel_coords' : to_numpy(input_dict['batch_dict']['voxel_coords']),
}
ort_outs = ort_session.run(None, ort_inputs)
ort_outs[0].shape


(962, 192)

In [18]:
ort_outs

[array([[-1.0968710e+00, -8.8788003e-02,  1.5693261e-01, ...,
         -1.0477213e+00,  3.7999816e-02, -1.9859798e-01],
        [-1.0964003e+00, -7.0831411e-02,  1.6052194e-01, ...,
         -1.0823513e+00,  4.2363975e-02, -1.9235148e-01],
        [-1.0862060e+00, -1.0094207e-01,  1.7857143e-01, ...,
         -1.0550426e+00,  2.6312700e-02, -1.5948443e-01],
        ...,
        [-3.6077169e-01,  2.1283090e-01, -1.5834051e-01, ...,
         -1.2332108e+00, -1.6346724e-01,  1.6772318e-01],
        [-6.1859196e-01,  3.5161918e-01, -1.1129949e-01, ...,
         -1.3723180e+00, -1.0930919e-01, -5.4504383e-02],
        [-6.1339909e-01,  3.2127568e-01, -1.2228360e-02, ...,
         -1.2449126e+00,  6.9372042e-04, -1.3799112e-01]], dtype=float32),
 array([1.], dtype=float32),
 array(1, dtype=int64),
 array([[-1.0968710e+00, -8.8788003e-02,  1.5693261e-01, ...,
         -1.0477213e+00,  3.7999816e-02, -1.9859798e-01],
        [-1.0964003e+00, -7.0831411e-02,  1.6052194e-01, ...,
         -1.082